<a href="https://colab.research.google.com/github/nikhilnair31/RM294---Optimatization-1-Group-Project-1/blob/main/RM294_Optimization_1_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[Optimisation Problem](https://utexas.instructure.com/courses/1369579/assignments/6461290)**

To formulate the marketing budget allocation problem as a linear program (LP), we need to define decision variables, objective function, and constraints. Let's denote the budget allocation for each medium as follows:

***Variables***:

$x_1$: Budget allocated to Print. </br>
$x_2$: Budget allocated to TV. </br>
$x_3$: Budget allocated to SEO. </br>
$x_4$: Budget allocated to AdWords.</br>
$x_5$: Budget allocated to Facebook.</br>
$x_6$: Budget allocated to LinkedIn.</br>
$x_7$: Budget allocated to Instagram.</br>
$x_8$: Budget allocated to Snapchat.</br>
$x_9$: Budget allocated to Twitter.</br>
$x_{10}$: Budget allocated to Email.</br>

***Objective Function***:
The objective is to maximize the overall ROI: </br>
               <h3><center> $ Maximize &nbsp; &nbsp; ROI_i * x_i \quad \text{for all } i = 1, 2, \ldots, 10\$</center></h3>

***Constraints***:
1. **Budget Constraint**:  $\sum_{i=1}^{10} x_i \leq 10\$
    
2. **TV and Print Constraint**: $x_1 + x_2 \leq x_5 + x_{10}$

3. **Social media and SEO/AdWords Constraint**: $\sum_{i=5}^{9} x_i \geq 2*(x_3 + x_4)\$

4. **Individual Budget Constraints**: $x_i \leq  3 \quad \text{for all } i = 1, 2, \ldots, 10\$

4. **Positivity Constraints**: $x_i \geq 0 \quad \text{for all } i = 1, 2, \ldots, 10\$

## Setup

In [1]:
!pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 21.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import gurobipy as gp
import pandas as pd

## Data Loading

In [3]:
# Loading ROI data in CSV
roi_csv_filepath = r'/content/drive/MyDrive/Colab Notebooks/2. Fall/O1/Data/ROI_data.csv'
roi_csv_file_df = pd.read_csv(roi_csv_filepath)
roi_csv_file_df

,Platform,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,ROI,0.031,0.049,0.024,0.039,0.016,0.024,0.046,0.026,0.033,0.044
1,Second Firms ROI Estimate,0.049,0.023,0.024,0.039,0.044,0.046,0.026,0.019,0.037,0.026


## Question 1, 2, 3 - Estimates from 1st Firm

In [4]:
# Create a Gurobi model
ojMod1 = gp.Model("Budget_Allocation")

Restricted license - for non-production use only - expires 2024-10-28


In [7]:
# Defining decision variables
roi_values_1 = roi_csv_file_df.iloc[0, :].tolist()[1:]
num_vars = len(roi_values_1)
ojMod1X = ojMod1.addMVar(num_vars, name="x")

In [9]:
# Setting the objective function
obj_coeffs1 = roi_values_1
ojMod1.setObjective(gp.quicksum(obj_coeffs1[i] * ojMod1X[i] for i in range(num_vars)), sense=gp.GRB.MAXIMIZE)

In [12]:
# Create variable names by taking column names
x_names = roi_csv_file_df.columns.tolist()[1:]

# Adding Budget constraint using index since all should have <= 10M constraint anyways
ojMod1.addConstr(gp.quicksum(ojMod1X[i] for i in range(num_vars)) <= 10, name="Budget_Constr")

# Add constraints using variable names assigned by column names
ojMod1.addConstr(ojMod1X[x_names.index("Print")] + ojMod1X[x_names.index("TV")] <= ojMod1X[x_names.index("Facebook")] + ojMod1X[x_names.index("Email")], name="Print_TV_Constr")
ojMod1.addConstr(ojMod1X[x_names.index("Facebook")] + ojMod1X[x_names.index("LinkedIn")] + ojMod1X[x_names.index("Instagram")] + ojMod1X[x_names.index("Snapchat")] + ojMod1X[x_names.index("Twitter")] >= 2 * (ojMod1X[x_names.index("SEO")] + ojMod1X[x_names.index("AdWords")]), name="Social_Media_Constr")

# Again adding Max Alloc constraint using index since all should have <= 3M constraint anyways
for i in range(num_vars):
    ojMod1.addConstr(ojMod1X[i] <= 3, name=f"Max_Alloc_Constraint_{i}")

In [13]:
ojMod1.Params.OutputFlag = 0
ojMod1.optimize()

In [14]:
ojMod1.objVal

0.45600000000000007

In [15]:
ojMod1X.x

array([0., 3., 0., 1., 0., 0., 3., 0., 0., 3.])

In [16]:
budget_alloc_1 = [ojMod1X[i].x for i in range(num_vars)]
var_names = roi_csv_file_df.columns.tolist()[1:]

df_1st = pd.DataFrame({
    "Channel": var_names,
    "Budget Alloc ($M)": budget_alloc_1
})

# Calculate the total budget and total ROI
budget_1 = sum(budget_alloc_1)
roi_1 = sum(obj_coeffs1[i] * budget_alloc_1[i] for i in range(num_vars))

# Print the DataFrame
print(f"1st Firm's Optimal Budget Allocation\n{df_1st}")
print(f"{'Total Budget':25} ${budget_1:.3f}M")
print(f"{'Total ROI':25} ${roi_1:.3f}M")

1st Firm's Optimal Budget Allocation
     Channel Budget Alloc ($M)
0      Print               0.0
1         TV               3.0
2        SEO               0.0
3    AdWords               1.0
4   Facebook               0.0
5   LinkedIn               0.0
6  Instagram               3.0
7   Snapchat               0.0
8    Twitter               0.0
9      Email               3.0
Total Budget              $10.000M
Total ROI                 $0.456M


## Question 4 - Estimates from 2nd Firm

In [ ]:
# Create a Gurobi model
ojMod2 = gp.Model("Budget_Allocation")

In [ ]:
# Define decision variables
roi_values_2 = roi_csv_file_df.iloc[1, -10:].tolist()
num_vars = len(roi_values_1)
ojMod2X = ojMod2.addMVar(num_vars, name="x")

In [ ]:
# Set the objective function
obj_coeffs2 = roi_csv_file_df.iloc[1, -10:].tolist()
ojMod2.setObjective(gp.quicksum(obj_coeffs2[i] * ojMod2X[i] for i in range(num_vars)), sense=gp.GRB.MAXIMIZE)

In [ ]:
ojMod2.addConstr(gp.quicksum(ojMod2X[i] for i in range(num_vars)) <= 10, name="Budget_Constr")
ojMod2.addConstr(ojMod2X[0] + ojMod2X[1] <= ojMod2X[4] + ojMod2X[9], name="Print_TV_Constr")
ojMod2.addConstr(gp.quicksum(ojMod2X[i] for i in range(4, 9)) >= 2*(ojMod2X[2] + ojMod2X[3]), name="Social_Media_Constr")
for i in range(num_vars):
    ojMod2.addConstr(ojMod2X[i] <= 3, name=f"Budget_Constraint_{i}")

In [ ]:
ojMod2.Params.OutputFlag = 0
ojMod2.optimize()

In [ ]:
ojMod2.objVal

0.45600000000000007

In [ ]:
ojMod2X.x

array([3., 0., 0., 1., 3., 3., 0., 0., 0., 0.])

In [ ]:
budget_alloc_2 = [ojMod2X[i].x for i in range(num_vars)]
var_names = roi_csv_file_df.columns.tolist()[1:]

df_2nd = pd.DataFrame({
    "Channel": var_names,
    "Budget Alloc ($M)": budget_alloc_2
})

# Calculate the total budget and total ROI
budget_2 = sum(budget_alloc_2)
roi_2 = sum(obj_coeffs2[i] * budget_alloc_2[i] for i in range(num_vars))

# Print the DataFrame
print(f"2nd Firm's Optimal Budget Allocation\n{df_2nd}")
print(f"{'Total Budget':25} ${budget_2:.3f}M")
print(f"{'Total ROI':25} ${roi_2:.3f}M")

2nd Firm's Optimal Budget Allocation
     Channel Budget Alloc ($M)
0      Print               3.0
1         TV               0.0
2        SEO               0.0
3    AdWords               1.0
4   Facebook               3.0
5   LinkedIn               3.0
6  Instagram               0.0
7   Snapchat               0.0
8    Twitter               0.0
9      Email               0.0
Total Budget              $10.000M
Total ROI                 $0.456M


## Question 5 - Allocation Comparison, Combining ROIs and Allocations, Checking for 3rd Constraints Significance

### Comparison

In [ ]:
df_1st_and_2nd = pd.merge(df_1st, df_2nd, on='Channel', suffixes=('_df1', '_df2'))
df_1st_and_2nd.rename(
    columns={
      'Budget Alloc ($M)_df1': 'Budget Alloc ($M) 1st Firm',
      'Budget Alloc ($M)_df2': 'Budget Alloc ($M) 2nd Firm'
    },
    inplace=True
)

df_1st_and_2nd['ROI 1st Firm'] = obj_coeffs1
df_1st_and_2nd['ROI 2nd Firm'] = obj_coeffs2

df_1st_and_2nd = df_1st_and_2nd[['Channel', 'ROI 1st Firm', 'Budget Alloc ($M) 1st Firm', 'ROI 2nd Firm', 'Budget Alloc ($M) 2nd Firm']]
df_1st_and_2nd

,Channel,ROI 1st Firm,Budget Alloc ($M) 1st Firm,ROI 2nd Firm,Budget Alloc ($M) 2nd Firm
0,Print,0.031,0.0,0.049,3.0
1,TV,0.049,3.0,0.023,0.0
2,SEO,0.024,0.0,0.024,0.0
3,AdWords,0.039,1.0,0.039,1.0
4,Facebook,0.016,0.0,0.044,3.0
5,LinkedIn,0.024,0.0,0.046,3.0
6,Instagram,0.046,3.0,0.026,0.0
7,Snapchat,0.026,0.0,0.019,0.0
8,Twitter,0.033,0.0,0.037,0.0
9,Email,0.044,3.0,0.026,0.0


> The two budget allocations are different but yield the same optimal value



### Combinations

In [ ]:
# Multiply 'ROI 1st Firm' by 'Budget Alloc ($M) 2nd Firm' and sum the results
df_1st_and_2nd['1st_roi_2nd_alloc'] = df_1st_and_2nd['ROI 1st Firm'] * df_1st_and_2nd['Budget Alloc ($M) 2nd Firm']
roi_1st_alloc_2nd = df_1st_and_2nd['1st_roi_2nd_alloc'].sum()
print(f"1st Firm's ROI with 2nd Firm's Allocation: ${roi_1st_alloc_2nd:.3f}M")
print(f"1st Firm's ROI with 1st Firm's Allocation: ${roi_1:.3f}M")
print(f"Difference in ROI: ${(roi_1-roi_1st_alloc_2nd):.3f}M")

1st Firm's ROI with 2nd Firm's Allocation: $0.252M
1st Firm's ROI with 1st Firm's Allocation: $0.456M
Difference in ROI: $0.204M


In [ ]:
# Multiply 'ROI 1st Firm' by 'Budget Alloc ($M) 2nd Firm' and sum the results
df_1st_and_2nd['2nd_roi_1st_alloc'] = df_1st_and_2nd['ROI 2nd Firm'] * df_1st_and_2nd['Budget Alloc ($M) 1st Firm']
roi_2nd_alloc_1st = df_1st_and_2nd['2nd_roi_1st_alloc'].sum()
print(f"2nd Firm's ROI with 1st Firm's Allocation: ${roi_2nd_alloc_1st:.3f}M")
print(f"2nd Firm's ROI with 2nd Firm's Allocation: ${roi_2:.3f}M")
print(f"Difference in ROI: ${(roi_2-roi_2nd_alloc_1st):.3f}M")

2nd Firm's ROI with 1st Firm's Allocation: $0.264M
2nd Firm's ROI with 2nd Firm's Allocation: $0.456M
Difference in ROI: $0.192M


### 3rd Constraint Significance

In [ ]:
# Create a Gurobi model
ojMod3 = gp.Model("Budget_Allocation")

In [ ]:
# Define decision variables
roi_values_1 = roi_csv_file_df.iloc[0, -10:].tolist()
num_vars = len(roi_values_1)

ojMod3X = ojMod3.addMVar(num_vars, name="x")

In [ ]:
# Set the objective function
obj_coeffs3 = roi_values_1
ojMod3.setObjective(gp.quicksum(obj_coeffs3[i] * ojMod3X[i] for i in range(num_vars)), sense=gp.GRB.MAXIMIZE)

In [ ]:
ojMod3.addConstr(gp.quicksum(ojMod3X[i] for i in range(num_vars)) <= 10, name="Budget_Constr")
ojMod3.addConstr(ojMod3X[0] + ojMod3X[1] <= ojMod3X[4] + ojMod3X[9], name="Print_TV_Constr")
ojMod3.addConstr(gp.quicksum(ojMod3X[i] for i in range(4, 9)) >= 2*(ojMod3X[2] + ojMod3X[3]), name="Social_Media_Constr")

<MConstr () *awaiting model update*>

In [ ]:
ojMod3.Params.OutputFlag = 0
ojMod3.optimize()

In [ ]:
print(f'{ojMod3.objVal}')

0.46499999999999997


In [ ]:
print(f'{ojMod3X.x}')

[0. 5. 0. 0. 0. 0. 0. 0. 0. 5.]


In [ ]:
budget_alloc_3 = [ojMod3X[i].x for i in range(num_vars)]
var_names = roi_csv_file_df.columns.tolist()[1:]

df_3rd = pd.DataFrame({
    "Channel": var_names,
    "Budget Alloc ($M)": budget_alloc_3
})

# Calculate the total budget and total ROI
budget_3 = sum(budget_alloc_3)
roi_3 = sum(obj_coeffs3[i] * budget_alloc_3[i] for i in range(num_vars))

# Print the DataFrame
print(f"1st Firm's Optimal Budget Allocation\n{df_3rd}")
print(f"{'Total Budget':25} ${budget_3:.3f}M")
print(f"{'Total ROI':25} ${roi_3:.3f}M")

1st Firm's Optimal Budget Allocation
     Channel Budget Alloc ($M)
0      Print               0.0
1         TV               5.0
2        SEO               0.0
3    AdWords               0.0
4   Facebook               0.0
5   LinkedIn               0.0
6  Instagram               0.0
7   Snapchat               0.0
8    Twitter               0.0
9      Email               5.0
Total Budget              $10.000M
Total ROI                 $0.465M




> Ignoring the third condition, yields different allocations for 1st firm but with a slightly different the Optimal ROI which causes investments to be focussed and not diversified like earlier allocations



## Question 6 - Channel Slack

In [ ]:
var_names = roi_csv_file_df.columns.tolist()[1:]
roi_values_1 = roi_csv_file_df.iloc[0, -10:].tolist()

In [ ]:
slack_df = pd.DataFrame()
slack_df['Channel'] = var_names
slack_df['Min ROI'] = ojMod1.SAObjLow
slack_df['Curr ROI'] = roi_values_1
slack_df['Max ROI'] = ojMod1.SAObjUp
slack_df

,Channel,Min ROI,Curr ROI,Max ROI
0,Print,-inf,0.031,0.049
1,TV,0.039,0.049,0.062
2,SEO,-inf,0.024,0.039
3,AdWords,0.033,0.039,0.046
4,Facebook,-inf,0.016,0.029
5,LinkedIn,-inf,0.024,0.039
6,Instagram,0.039,0.046,inf
7,Snapchat,-inf,0.026,0.039
8,Twitter,-inf,0.033,0.039
9,Email,0.029,0.044,inf


## Question 7 - Monthly ROI

In [ ]:
# Reading ROI by month by channel data from CSV
df_month_channel_roi = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/2. Fall/O1/Data/roi_mat.csv')
# Drop Unnamed column
df_month_channel_roi.rename(columns={'Unnamed: 0':'Months'}, inplace=True)
# Convert % to decimal
df_month_channel_roi[var_names] = df_month_channel_roi[var_names].apply(lambda x : x/100)

df_month_channel_roi

,Months,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,January,0.040,0.036,0.024,0.039,0.030,0.035,0.036,0.0225,0.035,0.035
1,February,0.040,0.039,0.027,0.038,0.043,0.032,0.027,0.0180,0.037,0.035
2,March,0.035,0.029,0.031,0.038,0.024,0.041,0.037,0.0260,0.042,0.025
3,April,0.038,0.031,0.024,0.044,0.024,0.038,0.037,0.0250,0.036,0.029
4,May,0.035,0.032,0.019,0.034,0.027,0.027,0.039,0.0220,0.045,0.039
5,June,0.040,0.032,0.027,0.034,0.034,0.030,0.045,0.0210,0.038,0.041
6,July,0.039,0.036,0.020,0.044,0.039,0.037,0.043,0.0180,0.040,0.038
7,August,0.042,0.033,0.028,0.042,0.020,0.037,0.036,0.0150,0.044,0.043
8,September,0.041,0.028,0.025,0.042,0.029,0.037,0.028,0.0250,0.040,0.034
9,October,0.030,0.030,0.031,0.046,0.031,0.033,0.032,0.0230,0.025,0.032


In [ ]:
# List of months
months = df_month_channel_roi['Months'].tolist()
var_names = roi_csv_file_df.columns.tolist()[1:]

# Initialize an empty list to store the budget allocation for each month
monthly_budget_allocations = []
# Initialize a variable to keep track of the cumulative return
cumulative_return = 0

In [ ]:
for month in months:
    # Convert the values in rois to numeric values (excluding the 'Month' column)
    rois = df_month_channel_roi.loc[months.index(month)].values[1:]
    rois = [float(value) for value in rois]

    # Calculate the budget for the current month, including reinvestment
    budget = 10 + cumulative_return

    # Create a new model for each month
    model = gp.Model("Budget_Allocation")

    modelX = model.addVars(len(var_names), name="x")

    # Define the objective function for the current month using rois
    model.setObjective(gp.quicksum(rois[i] * modelX[i] for i in range(len(var_names))), gp.GRB.MAXIMIZE)

    # Add the constraints as before
    model.addConstr(gp.quicksum(modelX[i] for i in range(num_vars)) <= budget, name="Budget_Constr")
    model.addConstr(modelX[0] + modelX[1] <= modelX[4] + modelX[9], name="Print_TV_Constr")
    model.addConstr(gp.quicksum(modelX[i] for i in range(4, 9)) >= 2*(modelX[2] + modelX[3]), name="Social_Media_Constr")
    for i in range(num_vars):
        model.addConstr(modelX[i] <= 3, name=f"Budget_Constraint_{i}")

    # Set the output flag to 0 to suppress Gurobi's output. Optimize the model for the current month
    model.Params.OutputFlag = 0
    model.optimize()

    # Calculate the return for the current month
    current_month_return = sum(rois[i] * modelX[i].x for i in range(len(var_names)))
    # Calculate the reinvestment amount (half of the return)
    reinvestment_amount = current_month_return * 0.5
    # Add the reinvestment amount to the cumulative return
    cumulative_return += reinvestment_amount

    # Store the budget allocation for the current month
    budget_allocation = [month] + [v.x for v in model.getVars()]
    monthly_budget_allocations.append(budget_allocation)

In [ ]:
df_allocations = pd.DataFrame(monthly_budget_allocations, columns=['Month'] + var_names).T

df_allocations.columns = df_allocations.iloc[0]
df_allocations = df_allocations.apply(lambda x: pd.to_numeric(x, errors='coerce').round(3))

df_allocations = df_allocations[1:]
df_allocations

Month,January,February,March,April,May,June,July,August,September,October,November,December
Print,3.000,3.000,0.00,0.000,1.804,3.00,1.124,3.000,1.363,0.000,3.000,3.000
TV,0.000,0.000,0.00,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,3.000
SEO,0.000,0.000,0.00,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000
AdWords,1.333,2.396,3.00,3.000,0.000,0.00,3.000,1.827,3.000,3.000,2.056,0.428
Facebook,0.000,3.000,0.00,0.000,0.000,0.00,1.124,0.000,0.000,0.000,0.000,3.000
LinkedIn,0.000,0.000,3.00,3.000,0.000,0.00,0.000,0.655,3.000,3.000,1.113,0.000
Instagram,2.667,0.000,1.39,3.000,3.000,3.00,3.000,0.000,0.000,3.000,3.000,0.000
Snapchat,0.000,0.000,0.00,0.000,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000
Twitter,0.000,1.791,3.00,1.597,3.000,2.02,3.000,3.000,3.000,0.000,0.000,0.000
Email,3.000,0.000,0.00,0.000,3.000,3.00,0.000,3.000,1.363,2.955,3.000,3.000


## Question 8 - Stable Budget

In [ ]:
df_allocations_new = df_allocations.copy()

for col in df_allocations.columns[1:]:
    df_allocations_new[col] = df_allocations[col] - df_allocations[df_allocations.columns[df_allocations.columns.get_loc(col) - 1]]

df_allocations_new['January'] = 0

df_allocations_new

Month,January,February,March,April,May,June,July,August,September,October,November,December
Print,0,0.000,-3.000,0.000,1.804,1.196,-1.876,1.876,-1.637,-1.363,3.000,0.000
TV,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000
SEO,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
AdWords,0,1.063,0.604,0.000,-3.000,0.000,3.000,-1.173,1.173,0.000,-0.944,-1.628
Facebook,0,3.000,-3.000,0.000,0.000,0.000,1.124,-1.124,0.000,0.000,0.000,3.000
LinkedIn,0,0.000,3.000,0.000,-3.000,0.000,0.000,0.655,2.345,0.000,-1.887,-1.113
Instagram,0,-2.667,1.390,1.610,0.000,0.000,0.000,-3.000,0.000,3.000,0.000,-3.000
Snapchat,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Twitter,0,1.791,1.209,-1.403,1.403,-0.980,0.980,0.000,0.000,-3.000,0.000,0.000
Email,0,-3.000,0.000,0.000,3.000,0.000,-3.000,3.000,-1.637,1.592,0.045,0.000


> The budget isn't stable as we can see an allocation difference between months across channels which is greater than the $1M limit.